In [14]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [56]:
from policyengine_us import Simulation


situation = {
    "people": {
        "you": {
            "age": {"2025": 40, "2026": 40},
            "capital_gains_before_response": {"2025": 200000, "2026": 200000},
            "real_estate_taxes": {"2025": 50000, "2026": 50000},
        },
        "your partner": {"age": {"2025": 40, "2026": 40}},
        "your first dependent": {"age": {"2025": 10, "2026": 10}},
        "your second dependent": {"age": {"2025": 10, "2026": 10}},
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ]
        }
    },
    "marital_units": {
        "your marital unit": {"members": ["you", "your partner"]},
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2025": 1, "2026": 1},
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2025": 2, "2026": 2},
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ]
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent",
            ],
            "state_name": {"2025": "CA", "2026": "CA"},
        }
    },
    "axes": [[{"name": "employment_income", "count": 300, "min": 0, "max": 300000}]],
}

simulation = Simulation(
    situation=situation,
)

# Calculate for both 2025 and 2026
simulation = Simulation(situation=situation)

# Calculate both metrics for each year
results = {}
for year in [2025, 2026]:
    results[year] = {
        "salt_deduction": simulation.calculate("salt_deduction", year),
        "alternative_minimum_tax": simulation.calculate(
            "alternative_minimum_tax", year
        ),
    }

In [57]:
# Create separate DataFrames for each year
for year in [2025, 2026]:
    df = pd.DataFrame(
        {
            "employment_income": range(0, 300000, 1000),
            "salt_deduction": results[year]["salt_deduction"],
            "alternative_minimum_tax": results[year]["alternative_minimum_tax"],
        }
    )

    # Save to CSV with year in filename
    df.to_csv(f"../data/tax_calculations_{year}.csv", index=False)

In [58]:
from policyengine_us import Simulation
from personal_calculator.calculator import calculate_impacts


def calculate_subsidy_rate(situation, period):
    """Calculate the marginal subsidy rate for real estate taxes under current law"""
    # Calculate baseline net income
    results = calculate_impacts(situation, {})
    current_law_base = results["current_law"]

    # Create modified situation with increased real estate taxes
    modified_situation = situation.copy()
    delta = 500.0  # $500 increment in real estate taxes

    # Modify the real estate taxes
    head_key = "you"
    if "real_estate_taxes" in modified_situation["people"][head_key]:
        current_taxes = modified_situation["people"][head_key]["real_estate_taxes"][
            period
        ]
        modified_situation["people"][head_key]["real_estate_taxes"][period] = (
            current_taxes + delta
        )

    # Calculate modified net income
    mod_results = calculate_impacts(modified_situation, {})
    current_law_mod = mod_results["current_law"]

    # Calculate and return subsidy rate
    return float(current_law_mod - current_law_base) / delta


# Calculate subsidy rates across earnings range
results = []
employment_incomes = range(0, 300000, 1000)

for income in employment_incomes:
    test_situation = situation.copy()
    test_situation["people"]["you"]["employment_income"] = {"2025": income}

    subsidy_2025 = calculate_subsidy_rate(test_situation, "2025")
    subsidy_2026 = calculate_subsidy_rate(test_situation, "2026")

    results.append(
        {
            "employment_income": income,
            "subsidy_rate_2025": subsidy_2025,
            "subsidy_rate_2026": subsidy_2026,
        }
    )

# Create DataFrame and save to CSV
df = pd.DataFrame(results)
df.to_csv("../data/subsidy_rates.csv", index=False)